# Grad-CAM

In [ ]:
# Import

import os
import numpy as np
import cv2  # img
import torch    # grad cam
import torchvision.transforms as transforms # grad cam
from torchvision import models  # grad cam
from pathlib import Path    # path iteration


In [ ]:

# print layer
# for name, module in model.named_modules():
#     print(name, "->", module.__class__.__name__)


In [4]:
# Define functions

# Load img and normalize
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (224, 224))  # Resize
    img = transforms.ToTensor()(img)
    img = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)
    img = img.unsqueeze(0)  # Add batch dimension
    return img

# get class label
def get_class_label(preds):
    _, class_index = torch.max(preds, 1)
    return class_index.item()

# get conv layer
def get_conv_layer(model, conv_layer_name):
    for name, layer in model.named_modules():
        if name == conv_layer_name:
            return layer
    raise ValueError(f"Layer '{conv_layer_name}' not found in the model.")

# get Grad-CAM heatmap
def compute_gradcam(model, img_tensor, class_index, conv_layer_name="layer4.2.conv2"):
    # get Conv-Layer
    conv_layer = get_conv_layer(model, conv_layer_name)

    # Forward- und Backward-data
    activations = None
    gradients = None

    # Forward-Hook: save Feature Maps (Activations)
    def forward_hook(module, input, output):
        nonlocal activations
        activations = output.detach()

    # Full-Backward-Hook: save Gradienten, Feature Maps
    def backward_hook(module, grad_input, grad_output):
        nonlocal gradients
        gradients = grad_output[0].detach()

    # register hooks
    fwd_hook = conv_layer.register_forward_hook(forward_hook)
    bwd_hook = conv_layer.register_full_backward_hook(backward_hook)

    # forwardpass
    preds = model(img_tensor)
    loss = preds[:, class_index]

    # Backpropagation
    model.zero_grad()
    loss.backward()

    # rm Hooks
    fwd_hook.remove()
    bwd_hook.remove()

    # Feature Maps & Gradients in NumPy
    activations = activations.cpu().numpy()[0]   # [C, H, W]
    gradients = gradients.cpu().numpy()[0]       # [C, H, W]

    # mean gradients -> weights
    pooled_grads = np.mean(gradients, axis=(1, 2))

    # combine weights
    for i in range(pooled_grads.shape[0]):
        activations[i, ...] *= pooled_grads[i]

    # calc heatmap
    heatmap = np.mean(activations, axis=0)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + 1e-8

    return heatmap

# Overlay heatmap on image
def overlay_heatmap(img_path, heatmap, alpha=0.5):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(img, alpha, heatmap, 1 - alpha, 0)
    return superimposed_img


In [ ]:
if __name__ == "__main__":
    
    # Load a pretrained model
    model = models.resnext50_32x4d()
    model.fc = torch.nn.Linear(model.fc.in_features, 15)
    model.load_state_dict(torch.load("../X_AI/trained_model.pth", map_location=torch.device('cpu'), weights_only=True))
    model.eval()    # activate evaluation (not training)

    # set paths
    input_root = Path("../X_AI/eval_data")
    output_root = Path("./gradcam_outputs")

    for filepath in input_root.rglob("*.jpg"):
        
        # path stuff
        subfolder = filepath.parent.name  
        save_dir = output_root / subfolder
        save_dir.mkdir(parents=True, exist_ok=True)
        
        # img to tensor
        img_tensor = preprocess_image(str(filepath))   

        # Get model predictions
        with torch.no_grad():
            preds = model(img_tensor)
        class_index = get_class_label(preds)
        # print(f"Predicted Class Index: {class_index}")

        # Compute Grad-CAM heatmap (last layer)
        heatmap = compute_gradcam(model, img_tensor, class_index, conv_layer_name = "layer4.2.conv2")

        # Overlay heatmap on the original image
        output_img = overlay_heatmap(filepath, heatmap)

        # Save the heatmap
        save_path = save_dir / filepath.name
        cv2.imwrite(str(save_path), output_img)
        
        print("Saved:", save_path)


Gespeichert: gradcam_outputs\BAS\0.jpg
Gespeichert: gradcam_outputs\BAS\1.jpg
Gespeichert: gradcam_outputs\BAS\10.jpg
Gespeichert: gradcam_outputs\BAS\11.jpg
Gespeichert: gradcam_outputs\BAS\12.jpg
Gespeichert: gradcam_outputs\BAS\13.jpg
Gespeichert: gradcam_outputs\BAS\14.jpg
Gespeichert: gradcam_outputs\BAS\15.jpg
Gespeichert: gradcam_outputs\BAS\16.jpg
Gespeichert: gradcam_outputs\BAS\2.jpg
Gespeichert: gradcam_outputs\BAS\3.jpg
Gespeichert: gradcam_outputs\BAS\4.jpg
Gespeichert: gradcam_outputs\BAS\5.jpg
Gespeichert: gradcam_outputs\BAS\6.jpg
Gespeichert: gradcam_outputs\BAS\7.jpg
Gespeichert: gradcam_outputs\BAS\8.jpg
Gespeichert: gradcam_outputs\BAS\9.jpg
Gespeichert: gradcam_outputs\EBO\0.jpg
Gespeichert: gradcam_outputs\EBO\1.jpg
Gespeichert: gradcam_outputs\EBO\10.jpg
Gespeichert: gradcam_outputs\EBO\11.jpg
Gespeichert: gradcam_outputs\EBO\12.jpg
Gespeichert: gradcam_outputs\EBO\13.jpg
Gespeichert: gradcam_outputs\EBO\14.jpg
Gespeichert: gradcam_outputs\EBO\15.jpg
Gespeichert: